# Coursera Capstone - The Battle of the Neighborhoods

### 1. Introduction

Paris is the most visited city in the whole world. Many tourists love it and many end up hating it! 

Being a multicultural city with each arrondissements having it's all peculiar characteristics. The ancient old question arise for every tourist planning to visit Paris, "Where should I stay?". 

The present report aims to analysing the diversity of each arrondissement. Leaving a better notion of where should tourists choose to stay accordingly to their preferences.

### 2. Business Problem

Thi report aims in to evaluate the best arrondissement for a tourist to stay accordingly to their trip preferences

### 3. Data Description

Geolocation data for Paris was obtained at https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e. 

Postal codes of the city serve as a starting point. Using Postal codes it's posible to find out the arrondissement, venues and their most popular venues categories.

The JSON file has data about all the neighbourhoods in France, limiting it to Paris.

postal_code : Postal codes for France
nom_comm : Name of Neighbourhoods in France
nom_dept : Name of the boroughs, equivalent to towns in France
geo_point_2d : Tuple containing the latitude and longitude of the Neighbourhoods

### 4. Metodology

First, all the packages must be imported and installed.

In [7]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install folium
import folium
from sklearn.cluster import KMeans

The the data was prepared and processed. The Paris map was plotted showing all of its arrondissements. Then, a model was build, clustering arrondissements by their characteristics, a new map showing the clusters was plotted.
After the process is done it's possible to analyse where a tourist should stay accordingly to his/her preference. 

#### 4.1. Data Collection

The data was collected the following link. Using the code bellow was posible to read the json data.

In [136]:
!wget -q -O 'france-data.json' https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
print("Data Downloaded!")

Data Downloaded!


In [137]:
paris_raw = pd.read_json('france-data.json')
paris_raw.head()

,datasetid,recordid,fields,geometry,record_timestamp
0,correspondances-code-insee-code-postal,2bf36b38314b6c39dfbcd09225f97fa532b1fc45,"{'code_comm': '645', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.2517129721...",2016-09-21T00:29:06.175+02:00
1,correspondances-code-insee-code-postal,7ee82e74e059b443df18bb79fc5a19b1f05e5a88,"{'code_comm': '133', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [3.0529405055...",2016-09-21T00:29:06.175+02:00
2,correspondances-code-insee-code-postal,e2cd3186f07286705ed482a10b6aebd9de633c81,"{'code_comm': '378', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.1971816504...",2016-09-21T00:29:06.175+02:00
3,correspondances-code-insee-code-postal,868bf03527a1d0a9defe5cf4e6fa0a730d725699,"{'code_comm': '243', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [2.7097808131...",2016-09-21T00:29:06.175+02:00
4,correspondances-code-insee-code-postal,21e809b1d4480333c8b6fe7addd8f3b06f343e2c,"{'code_comm': '003', 'nom_dept': 'VAL-DE-MARNE...","{'type': 'Point', 'coordinates': [2.3335102498...",2016-09-21T00:29:06.175+02:00


#### 4.2. Data Processing

In [138]:
paris_field_data = pd.DataFrame()
for f in paris_raw.fields:
    dict_new = f
    paris_field_data = paris_field_data.append(dict_new, ignore_index=True)

paris_field_data.head()

,code_arr,code_cant,code_comm,code_dept,code_reg,geo_point_2d,geo_shape,id_geofla,insee_com,nom_comm,nom_dept,nom_region,population,postal_code,statut,superficie,z_moyen
0,3,03,645,91,11,"[48.750443119964764, 2.251712972144151]","{'type': 'Polygon', 'coordinates': [[[2.238024...",16275,91645,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,15.5,91370,Commune simple,999.0,121.0
1,3,20,133,77,11,"[48.41256065214989, 3.052940505560729]","{'type': 'Polygon', 'coordinates': [[[3.076046...",31428,77133,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,0.2,77126,Commune simple,1082.0,88.0
2,1,09,378,91,11,"[48.52726809075556, 2.19718165044305]","{'type': 'Polygon', 'coordinates': [[[2.203466...",30975,91378,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,0.3,91730,Commune simple,313.0,150.0
3,5,14,243,77,11,"[48.87307018579678, 2.7097808131278462]","{'type': 'Polygon', 'coordinates': [[[2.727542...",17000,77243,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,20.2,77400,Chef-lieu canton,579.0,71.0
4,3,34,003,94,11,"[48.80588035965699, 2.333510249842654]","{'type': 'Polygon', 'coordinates': [[[2.343851...",32123,94003,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,19.5,94110,Chef-lieu canton,232.0,70.0


Selection of the collumns requiered (geo_point_2d, nom_dept, nom_comm and postal_code)

In [139]:
df = paris_field_data[['postal_code','nom_comm','nom_dept','geo_point_2d']]
df

,postal_code,nom_comm,nom_dept,geo_point_2d
0,91370,VERRIERES-LE-BUISSON,ESSONNE,"[48.750443119964764, 2.251712972144151]"
1,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,"[48.41256065214989, 3.052940505560729]"
2,91730,MAUCHAMPS,ESSONNE,"[48.52726809075556, 2.19718165044305]"
3,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,"[48.87307018579678, 2.7097808131278462]"
4,94110,ARCUEIL,VAL-DE-MARNE,"[48.80588035965699, 2.333510249842654]"
...,...,...,...,...
1295,77520,CESSOY-EN-MONTOIS,SEINE-ET-MARNE,"[48.50730730461658, 3.138844194183689]"
1296,93420,VILLEPINTE,SEINE-SAINT-DENIS,"[48.95902025378707, 2.536306342059409]"
1297,77130,CANNES-ECLUSE,SEINE-ET-MARNE,"[48.36403767307805, 2.990786679832767]"
1298,78930,VILLETTE,YVELINES,"[48.92627887061508, 1.6937417245662671]"


Selection of Paris data

In [140]:
df_paris = df[df['nom_dept'].str.contains('PARIS')].reset_index(drop=True)
df_paris

,postal_code,nom_comm,nom_dept,geo_point_2d
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]"
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]"
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]"
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]"
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,"[48.84896809191946, 2.332670898588416]"
5,75004,PARIS-4E-ARRONDISSEMENT,PARIS,"[48.854228281954754, 2.357361938142205]"
6,75001,PARIS-1ER-ARRONDISSEMENT,PARIS,"[48.8626304851685, 2.336293446550539]"
7,75017,PARIS-17E-ARRONDISSEMENT,PARIS,"[48.88733716648682, 2.307485559493426]"
8,75008,PARIS-8E-ARRONDISSEMENT,PARIS,"[48.87252726662346, 2.312582560420059]"
9,75013,PARIS-13E-ARRONDISSEMENT,PARIS,"[48.82871768452136, 2.362468228516128]"


#### 4.4 Geolocations in Paris

In [141]:
df_paris['geo_point_2d'][0]

[48.87689616237872, 2.337460241388529]

In [142]:
temp1 = df_paris['geo_point_2d']
temp1

0      [48.87689616237872, 2.337460241388529]
1      [48.86790337886785, 2.344107166658533]
2      [48.85941549762748, 2.378741060237548]
3      [48.86305413181178, 2.359361058970589]
4      [48.84896809191946, 2.332670898588416]
5     [48.854228281954754, 2.357361938142205]
6       [48.8626304851685, 2.336293446550539]
7      [48.88733716648682, 2.307485559493426]
8      [48.87252726662346, 2.312582560420059]
9      [48.82871768452136, 2.362468228516128]
10     [48.83515623066034, 2.419807034965275]
11    [48.844508659617546, 2.349859385560182]
12     [48.88686862295828, 2.384694327870042]
13     [48.86318677744551, 2.400819826729021]
14     [48.87602855694339, 2.361112904561707]
15     [48.86039876035177, 2.262099559395783]
16    [48.892735074561706, 2.348711933867703]
17     [48.85608259819694, 2.312438687733857]
18     [48.84015541860987, 2.293559372435076]
19     [48.82899321160942, 2.327100883257538]
Name: geo_point_2d, dtype: object

In [143]:
paris_latlng = df_paris['geo_point_2d'].astype('str')

Spliting the geo_point_2d column into latitude and longitude

#### Latitude

In [144]:
paris_lat = paris_latlng.apply(lambda x: x.split(',')[0])
paris_lat = paris_lat.apply(lambda x: x.lstrip('['))
paris_lat

0      48.87689616237872
1      48.86790337886785
2      48.85941549762748
3      48.86305413181178
4      48.84896809191946
5     48.854228281954754
6       48.8626304851685
7      48.88733716648682
8      48.87252726662346
9      48.82871768452136
10     48.83515623066034
11    48.844508659617546
12     48.88686862295828
13     48.86318677744551
14     48.87602855694339
15     48.86039876035177
16    48.892735074561706
17     48.85608259819694
18     48.84015541860987
19     48.82899321160942
Name: geo_point_2d, dtype: object

#### Longitude

In [145]:
paris_lng = paris_latlng.apply(lambda x: x.split(',')[1])
paris_lng = paris_lng.apply(lambda x: x.rstrip(']'))
paris_lng

0      2.337460241388529
1      2.344107166658533
2      2.378741060237548
3      2.359361058970589
4      2.332670898588416
5      2.357361938142205
6      2.336293446550539
7      2.307485559493426
8      2.312582560420059
9      2.362468228516128
10     2.419807034965275
11     2.349859385560182
12     2.384694327870042
13     2.400819826729021
14     2.361112904561707
15     2.262099559395783
16     2.348711933867703
17     2.312438687733857
18     2.293559372435076
19     2.327100883257538
Name: geo_point_2d, dtype: object

In [146]:
paris_geo_lat  = pd.DataFrame(paris_lat.astype(float))
paris_geo_lat.columns=['Latitude']
paris_geo_lng = pd.DataFrame(paris_lng.astype(float))
paris_geo_lng.columns=['Longitude']

In [147]:
paris_combined_data = pd.concat([df_paris.drop('geo_point_2d', axis=1), paris_geo_lat, paris_geo_lng], axis=1)
paris_combined_data

,postal_code,nom_comm,nom_dept,Latitude,Longitude
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671
5,75004,PARIS-4E-ARRONDISSEMENT,PARIS,48.854228,2.357362
6,75001,PARIS-1ER-ARRONDISSEMENT,PARIS,48.862630,2.336293
7,75017,PARIS-17E-ARRONDISSEMENT,PARIS,48.887337,2.307486
8,75008,PARIS-8E-ARRONDISSEMENT,PARIS,48.872527,2.312583
9,75013,PARIS-13E-ARRONDISSEMENT,PARIS,48.828718,2.362468


#### Coordinates

Getting the coodinates

In [148]:
paris = geocode(address='Paris, France, FR')[0]
paris_lng_coords = paris['location']['x']
paris_lat_coords = paris['location']['y']
print("The geolocation of Paris: ", paris_lat_coords, paris_lng_coords)

The geolocation of Paris:  48.85341000000005 2.3488000000000397


### Visualize the Map of Paris

In [149]:
map_Paris= folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)
map_Paris

for latitude, longitude, borough, town in zip(paris_combined_data['Latitude'], paris_combined_data['Longitude'], paris_combined_data['nom_comm'], paris_combined_data['nom_dept']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_opacity=0.8
        ).add_to(map_Paris)  
    
map_Paris

### Accessing Locations in Paris

In [150]:
# @hidden_cell

CLIENT_ID = 'K22GLKKK0V33YRZL2SFGJSKFRIXTCEVKTKFKWOKYMAOIRVX5' 
CLIENT_SECRET = 'SM3DGJR0RJ5U5UP0JGAQBIBOUTCU2RMED5SMFV4LWCEBBEEB'
VERSION = '20180605' # Foursquare API version

In [151]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [152]:
venues_in_Paris = getNearbyVenues(paris_combined_data['nom_comm'], paris_combined_data['Latitude'], paris_combined_data['Longitude'])

PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


In [153]:
venues_in_Paris.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,So Nat,Vegetarian / Vegan Restaurant
1,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,RAP,Gourmet Shop
2,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Le Bouclier de Bacchus,Wine Bar
3,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Farine & O,Bakery
4,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Place Saint-Georges,Plaza


### Grouping by Venues Categories

In [154]:
venues_in_Paris.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Afghan Restaurant,PARIS-11E-ARRONDISSEMENT,48.859415,2.378741,Afghanistan
African Restaurant,PARIS-9E-ARRONDISSEMENT,48.876896,2.361113,Wally Le Saharien
American Restaurant,PARIS-6E-ARRONDISSEMENT,48.892735,2.384694,Harper's
Antique Shop,PARIS-9E-ARRONDISSEMENT,48.876896,2.337460,Hôtel des Ventes Drouot
Argentinian Restaurant,PARIS-3E-ARRONDISSEMENT,48.863054,2.359361,Anahi
...,...,...,...,...
Wine Bar,PARIS-9E-ARRONDISSEMENT,48.892735,2.400820,Ô Château
Wine Shop,PARIS-3E-ARRONDISSEMENT,48.892735,2.361113,Trois Fois Vin
Women's Store,PARIS-2E-ARRONDISSEMENT,48.867903,2.344107,Promod


### One Hot Enconding

In [155]:
Paris_venue_cat = pd.get_dummies(venues_in_Paris[['Venue Category']], prefix="", prefix_sep="")
Paris_venue_cat

,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1276,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1277,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
Paris_venue_cat['Neighbourhood'] = venues_in_Paris['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [Paris_venue_cat.columns[-1]] + list(Paris_venue_cat.columns[:-1])
Paris_venue_cat = Paris_venue_cat[fixed_columns]

Paris_venue_cat.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
Paris_grouped = Paris_venue_cat.groupby('Neighbourhood').mean().reset_index()
Paris_grouped.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-10E-ARRONDISSEMENT,0.000000,0.02,0.0,0.0,0.0,0.0,0.000000,0.0,0.030000,...,0.0,0.0,0.01,0.0,0.000000,0.020000,0.02,0.0,0.0,0.0
1,PARIS-11E-ARRONDISSEMENT,0.025641,0.00,0.0,0.0,0.0,0.0,0.025641,0.0,0.025641,...,0.0,0.0,0.00,0.0,0.051282,0.025641,0.00,0.0,0.0,0.0
2,PARIS-12E-ARRONDISSEMENT,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.0,0.2,0.2
3,PARIS-13E-ARRONDISSEMENT,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.190476,...,0.0,0.0,0.00,0.0,0.206349,0.000000,0.00,0.0,0.0,0.0
4,PARIS-14E-ARRONDISSEMENT,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0


In [158]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [159]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [160]:
# create a new dataframe for Paris
neighborhoods_venues_sorted_paris = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_paris['Neighbourhood'] = Paris_grouped['Neighbourhood']

for ind in np.arange(Paris_grouped.shape[0]):
    neighborhoods_venues_sorted_paris.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_paris.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-10E-ARRONDISSEMENT,French Restaurant,Bistro,Hotel,Café,Coffee Shop,Pizza Place,Italian Restaurant,Indian Restaurant,Asian Restaurant,Burger Joint
1,PARIS-11E-ARRONDISSEMENT,Restaurant,French Restaurant,Café,Bar,Vietnamese Restaurant,Pastry Shop,Bakery,Italian Restaurant,Bistro,South American Restaurant
2,PARIS-12E-ARRONDISSEMENT,Zoo Exhibit,Supermarket,Monument / Landmark,Bistro,Zoo,Argentinian Restaurant,African Restaurant,Frozen Yogurt Shop,French Restaurant,Fountain
3,PARIS-13E-ARRONDISSEMENT,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Bus Stop,Plaza,Juice Bar,Sandwich Place,Farmers Market
4,PARIS-14E-ARRONDISSEMENT,French Restaurant,Hotel,Plaza,Food & Drink Shop,Bistro,Bakery,Sushi Restaurant,Laundromat,Brasserie,Italian Restaurant


### Model Bulding (K Means)

The number of clusters was choosen in order to make the analysis simpler.

In [161]:
# set number of clusters
k_num_clusters = 5

Paris_grouped_clustering = Paris_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_Paris = KMeans(n_clusters=k_num_clusters, random_state=0).fit(Paris_grouped_clustering)
kmeans_Paris

KMeans(n_clusters=5, random_state=0)

#### Labelling Clustered Data

In [162]:
kmeans_Paris.labels_

array([0, 0, 2, 4, 3, 0, 1, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 0],
      dtype=int32)

In [163]:
neighborhoods_venues_sorted_paris.insert(0, 'Cluster Labels', kmeans_Paris.labels_ +1)

In [164]:
paris_data = paris_combined_data

paris_data = paris_data.join(neighborhoods_venues_sorted_paris.set_index('Neighbourhood'), on='nom_comm')

paris_data.head()

,postal_code,nom_comm,nom_dept,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460,1,French Restaurant,Hotel,Bistro,Japanese Restaurant,Bakery,Wine Bar,Lounge,Cocktail Bar,Plaza,Gym / Fitness Center
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107,1,French Restaurant,Cocktail Bar,Wine Bar,Hotel,Italian Restaurant,Creperie,Bakery,Coffee Shop,Furniture / Home Store,Pedestrian Plaza
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741,1,Restaurant,French Restaurant,Café,Bar,Vietnamese Restaurant,Pastry Shop,Bakery,Italian Restaurant,Bistro,South American Restaurant
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361,1,French Restaurant,Coffee Shop,Japanese Restaurant,Art Gallery,Italian Restaurant,Bakery,Gourmet Shop,Burger Joint,Sandwich Place,Cocktail Bar
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671,1,Bakery,French Restaurant,Chocolate Shop,Plaza,Wine Bar,Fountain,Electronics Store,Bistro,Garden,Bookstore


In [165]:
paris_data_nonan = paris_data.dropna(subset=['Cluster Labels'])

### Map of Paris with arrondissements clustered

In [166]:
map_clusters_paris = folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_data_nonan['Latitude'], paris_data_nonan['Longitude'], paris_data_nonan['nom_comm'], paris_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + ' ' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.8
        ).add_to(map_clusters_paris)
        
map_clusters_paris

### 5. Results

Examining clusters 1 to 5 and obtaining results.

In [167]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 1, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-9E-ARRONDISSEMENT,1,French Restaurant,Hotel,Bistro,Japanese Restaurant,Bakery,Wine Bar,Lounge,Cocktail Bar,Plaza,Gym / Fitness Center
1,PARIS-2E-ARRONDISSEMENT,1,French Restaurant,Cocktail Bar,Wine Bar,Hotel,Italian Restaurant,Creperie,Bakery,Coffee Shop,Furniture / Home Store,Pedestrian Plaza
2,PARIS-11E-ARRONDISSEMENT,1,Restaurant,French Restaurant,Café,Bar,Vietnamese Restaurant,Pastry Shop,Bakery,Italian Restaurant,Bistro,South American Restaurant
3,PARIS-3E-ARRONDISSEMENT,1,French Restaurant,Coffee Shop,Japanese Restaurant,Art Gallery,Italian Restaurant,Bakery,Gourmet Shop,Burger Joint,Sandwich Place,Cocktail Bar
4,PARIS-6E-ARRONDISSEMENT,1,Bakery,French Restaurant,Chocolate Shop,Plaza,Wine Bar,Fountain,Electronics Store,Bistro,Garden,Bookstore
5,PARIS-4E-ARRONDISSEMENT,1,French Restaurant,Ice Cream Shop,Clothing Store,Park,Hotel,Plaza,Wine Bar,Pastry Shop,Pedestrian Plaza,Tea Room
11,PARIS-5E-ARRONDISSEMENT,1,French Restaurant,Hotel,Italian Restaurant,Plaza,Bakery,Café,Coffee Shop,Pub,Creperie,Vietnamese Restaurant
12,PARIS-19E-ARRONDISSEMENT,1,French Restaurant,Bar,Supermarket,Hotel,Café,Beer Bar,Seafood Restaurant,Bistro,Creperie,Sandwich Place
13,PARIS-20E-ARRONDISSEMENT,1,Bakery,Plaza,Bistro,French Restaurant,Japanese Restaurant,Italian Restaurant,Park,Hotel,Café,Bus Stop
14,PARIS-10E-ARRONDISSEMENT,1,French Restaurant,Bistro,Hotel,Café,Coffee Shop,Pizza Place,Italian Restaurant,Indian Restaurant,Asian Restaurant,Burger Joint


In [168]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 2, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,PARIS-16E-ARRONDISSEMENT,2,Plaza,Lake,Bus Stop,Pool,Park,French Restaurant,Boat or Ferry,Art Museum,Bus Station,Zoo Exhibit


In [169]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 3, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,PARIS-12E-ARRONDISSEMENT,3,Zoo Exhibit,Supermarket,Monument / Landmark,Bistro,Zoo,Argentinian Restaurant,African Restaurant,Frozen Yogurt Shop,French Restaurant,Fountain


In [170]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 4, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,PARIS-1ER-ARRONDISSEMENT,4,French Restaurant,Japanese Restaurant,Hotel,Plaza,Italian Restaurant,Art Museum,Tea Room,Historic Site,Garden,Udon Restaurant
7,PARIS-17E-ARRONDISSEMENT,4,Hotel,French Restaurant,Italian Restaurant,Bakery,Japanese Restaurant,Café,Restaurant,Plaza,Bistro,Burger Joint
8,PARIS-8E-ARRONDISSEMENT,4,French Restaurant,Hotel,Corsican Restaurant,Cocktail Bar,Spa,Theater,Plaza,Hotel Bar,Resort,Park
17,PARIS-7E-ARRONDISSEMENT,4,French Restaurant,Hotel,Café,Italian Restaurant,Plaza,Cocktail Bar,History Museum,Coffee Shop,Restaurant,Garden
19,PARIS-14E-ARRONDISSEMENT,4,French Restaurant,Hotel,Plaza,Food & Drink Shop,Bistro,Bakery,Sushi Restaurant,Laundromat,Brasserie,Italian Restaurant


In [171]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 5, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,PARIS-13E-ARRONDISSEMENT,5,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Bus Stop,Plaza,Juice Bar,Sandwich Place,Farmers Market


### 6. Discussion

Paris is a lively city. It has a wide variety of restaurants of different cusines. There are a lot of places to go and have some some fun such as restaurants Restaurants and Bars. Fo tourists Paris seems like a great city to visit with all of it's museums, parks, art galleries and history. Paris seems as a multicultural city ith lot of spots to go and have amusing trip.

If a tourist plans for a calm vacation with a lot of site seeing is better that he/she stays at the 16 arrondissement. 
On the other hand, If the tourist wants to be around many bars, and french  restaurants he/she should stay at  arrondissements: 1 - 6, 9 - 11, 15, 18 -20.
If the tourist has a strong preference for asian food he/she would prefer to stay at the 13 arrondissement.

### 7. Conclusion

Paris is a great city for tourists to visit with a lot of things to do and places to go. The cusine is rich and multidiverse, making it impossible to don't find something to the tourist taste.
Overall, Paris holds the title of the most visited city by tourits in the world for a great reason, all it's arrondissements present great options for diverse preferences.